# https://www.kaggle.com/c/mercari-price-suggestion-challenge 
# https://www.kaggle.com/c/home-credit-default-risk

Выбрать одно из двух соревнований. Выбирайте по данным, с которыми вам интереснее работать.

In [1]:
import time
import re
# from __future__ import print_function
from collections import defaultdict

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import pandas_profiling

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import make_union, make_pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler, LabelEncoder, MinMaxScaler,  Imputer, LabelBinarizer, OneHotEncoder
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV

# Ансамбли

import xgboost as xgb
import lightgbm as lgb

%matplotlib inline
plt.rcParams["figure.figsize"] = (15, 8)
pd.options.display.float_format = '{:.2f}'.format

In [2]:
df = pd.read_csv('train.tsv', sep='\t')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1482535 entries, 0 to 1482534
Data columns (total 8 columns):
train_id             1482535 non-null int64
name                 1482535 non-null object
item_condition_id    1482535 non-null int64
category_name        1476208 non-null object
brand_name           849853 non-null object
price                1482535 non-null float64
shipping             1482535 non-null int64
item_description     1482531 non-null object
dtypes: float64(1), int64(3), object(4)
memory usage: 90.5+ MB


train_id or test_id - the id of the listing

name - the title of the listing. Note that we have cleaned the data to remove text that look like prices (e.g. $20) to avoid leakage. These removed prices are represented as [rm]

item_condition_id - the condition of the items provided by the seller

category_name - category of the listing

brand_name

price - the price that the item was sold for. This is the target variable that you will predict. The unit is USD. This column doesn't exist in test.tsv since that is what you will predict.

shipping - 1 if shipping fee is paid by seller and 0 by buyer

item_description - the full description of the item. Note that we have cleaned the data to remove text that look like prices (e.g. $20) to avoid leakage. These removed prices are represented as [rm]


### Используйте параметр nrows, чтобы уменьшить выборку и сделать базовый разведочный анализ данных

In [4]:
df_p = pd.read_csv('train.tsv', nrows=50000, sep='\t')
df_p.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 8 columns):
train_id             50000 non-null int64
name                 50000 non-null object
item_condition_id    50000 non-null int64
category_name        49763 non-null object
brand_name           28416 non-null object
price                50000 non-null float64
shipping             50000 non-null int64
item_description     50000 non-null object
dtypes: float64(1), int64(3), object(4)
memory usage: 3.1+ MB


In [5]:
df.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.00,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.00,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.00,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.00,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.00,0,Complete with certificate of authenticity


In [6]:
!head sample_submission.csv

test_id,price
0,26.738
1,26.738
2,26.738
3,26.738
4,26.738
5,26.738
6,26.738
7,26.738
8,26.738


In [7]:
df_test = pd.read_csv('test.tsv', sep='\t')

In [8]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 693359 entries, 0 to 693358
Data columns (total 7 columns):
test_id              693359 non-null int64
name                 693359 non-null object
item_condition_id    693359 non-null int64
category_name        690301 non-null object
brand_name           397834 non-null object
shipping             693359 non-null int64
item_description     693359 non-null object
dtypes: int64(3), object(4)
memory usage: 37.0+ MB


In [9]:
df_test.head()

,test_id,name,item_condition_id,category_name,brand_name,shipping,item_description
0,0,"Breast cancer ""I fight like a girl"" ring",1,Women/Jewelry/Rings,NaN,1,Size 7
1,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers",1,Other/Office supplies/Shipping Supplies,NaN,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers Lined..."
2,2,Coach bag,1,Vintage & Collectibles/Bags and Purses/Handbag,Coach,1,Brand new coach bag. Bought for [rm] at a Coac...
3,3,Floral Kimono,2,Women/Sweaters/Cardigan,NaN,0,-floral kimono -never worn -lightweight and pe...
4,4,Life after Death,3,Other/Books/Religion & Spirituality,NaN,1,Rediscovering life after the loss of a loved o...


# Задание
1. Сделать baseline submission
  * Исследовать признак price. 
  * Исследовать признак price в зависимости от brand_name или других признаков
2. Реализовать цикл анализа
  * признаки -> модель -> настройка параметров -> лучшая модель и ее значение метрики качества на кросс-валидации
3. Провести простые преобразования признаков и добавление простых признаков
  * разделить category_name на уровни
  * посмотреть на наличие числовых значений в описании и имени
  * ...
4. Составить план по применению нескольких моделей на разных признаках
  * спроектируйте эксперимент. Нужно заранее спланировать порядок перебора признаков и моделей. Потом только писать код. Обратный порядок вызывает необходимость переписывать существующий код, это трата времени
5. Просмотреть 1-5 kernel на kaggle. (только в таком порядке. сначала работаем самостоятельно, потом смотрим идеи других. при появлении опыта можно сразу начинать с них)
6. Скорректировать план
  * убрать пункты, которые кажутся неудачными
  * добавить идеи из kernel, кажущиеся удачными
7. Построить модель, выбрать лучшую
8. Построить ансамбль, настрить парамертры. Сравнить с другими моделями.
9. Применить и засабмитить лучшую на cv модель
10. Прислать блокнот и свой ник в лидерборде

# Feature Engineering

In [10]:
pandas_profiling.ProfileReport(df_p)

Number of variables,8
Number of observations,50000
Total Missing (%),5.5%
Total size in memory,3.1 MiB
Average record size in memory,64.0 B
Numeric,3
Categorical,4
Boolean,1
Date,0
Text (Unique),0
Rejected,0


item_description и name практически уникальные, поэтому для начала нет смысла их учитывать для анализа

In [11]:
df_train = df_p.copy()

In [12]:
df_train.drop(columns=["item_description","name","train_id"],inplace=True)

In [13]:
df_train["brand_name"].fillna("unknown",inplace=True)
df_train["category_name"].fillna("unknown",inplace=True)
df_train.drop_duplicates(inplace=True)

In [14]:
df_train.head()

,item_condition_id,category_name,brand_name,price,shipping
0,3,Men/Tops/T-shirts,unknown,10.00,1
1,3,Electronics/Computers & Tablets/Components & P...,Razer,52.00,0
2,1,Women/Tops & Blouses/Blouse,Target,10.00,1
3,1,Home/Home Décor/Home Décor Accents,unknown,35.00,1
4,1,Women/Jewelry/Necklaces,unknown,44.00,0


In [15]:
pandas_profiling.ProfileReport(df_train)

Number of variables,6
Number of observations,37270
Total Missing (%),0.0%
Total size in memory,1.7 MiB
Average record size in memory,48.0 B
Numeric,3
Categorical,2
Boolean,1
Date,0
Text (Unique),0
Rejected,0


In [16]:
del df_p, df_train

Таким образом, остановимся на следущих признаках:

* shipping явно влияет на цену
* item_condition_id больше коррелирует с shipping, но может вносить уточнение в цену
* из опыта категория товаров явно влияет на цену
* так же из опыта, товар из одной категории, но с разным брендом оказывает влияние на цену (обувь addidas vs noname made in china)
* нам нужно предсказывать цену с какой-то погрешностью, таким образом выявление цены с точки зрения категорий не целесообразно. Цена так же имеет логарифмическую шкалу.

# Выбор моделей

Для начала предположим, что выбросов нет

В качестве метрики возьмем RMSE, чеем она ниже, тем лучше модель

In [18]:
from sklearn.pipeline import make_union, make_pipeline, Pipeline
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer

class LabelEncoderPipelineFriendly(LabelEncoder):
    
    def fit(self, X, y=None):
        """this would allow us to fit the model based on the X input."""
        super(LabelEncoderPipelineFriendly, self).fit(X)
        
    def transform(self, X, y=None):
        return super(LabelEncoderPipelineFriendly, self).transform(X).reshape(-1, 1)

    def fit_transform(self, X, y=None):
        return super(LabelEncoderPipelineFriendly, self).fit(X).transform(X).reshape(-1, 1)

def get_y(df):
    return np.log(df["price"]+1) # +1 чтобы не было log(0)

def get_nums(df):
    return df[["shipping","item_condition_id"]]

def get_cat_name(df):
    return df[["category_name"]]

def get_brand_name(df):
    return df[["brand_name"]]

vec = make_union(*[
    make_pipeline(FunctionTransformer(get_nums, validate=False)),
    make_pipeline(FunctionTransformer(get_cat_name, validate=False),SimpleImputer(strategy="constant",fill_value="unknown"),LabelEncoderPipelineFriendly()),
    make_pipeline(FunctionTransformer(get_brand_name, validate=False),SimpleImputer(strategy="constant",fill_value="unknown"),LabelEncoderPipelineFriendly()),
])

vec2 = make_union(*[
    make_pipeline(FunctionTransformer(get_nums, validate=False)),
    make_pipeline(FunctionTransformer(get_cat_name, validate=False),SimpleImputer(strategy="constant",fill_value="unknown"),OneHotEncoder(sparse=False)),
    make_pipeline(FunctionTransformer(get_brand_name, validate=False),SimpleImputer(strategy="constant",fill_value="unknown"),OneHotEncoder(sparse=False)),
])

In [28]:
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV, cross_val_score
from scipy.stats import randint,expon
from time import time
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import train_test_split

def report(search_res):
    print("Best param: {0}".format(search_res.best_params_))
    print("Best score: {0}".format(search_res.best_score_))
    
def test_model(model,v,df):
    y=get_y(df)
    X=v.fit_transform(df)
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    model.fit(X_train, y_train)
    print("RMSE:",mean_squared_error(model.predict(X_test), y_test))
    
def gs_model(model,param,v,df,n_iter_search = 20, cv=5, jobs=4, verbose=10):
    y=get_y(df)
    X=v.fit_transform(df)
#     rmse_scorer = make_scorer(mean_squared_error)
#     search = RandomizedSearchCV(model, param_distributions=param, scoring=rmse_scorer,
#                                    n_iter=n_iter_search, cv=cv, n_jobs=4)
    search = RandomizedSearchCV(model, param_distributions=param, scoring="neg_mean_squared_error",
                                   n_iter=n_iter_search, cv=cv, n_jobs=jobs, verbose=verbose, pre_dispatch=4)
#     search = RandomizedSearchCV(model, param_distributions=param, scoring="neg_mean_squared_error",
#                                    n_iter=n_iter_search, cv=cv, verbose=verbose)
#     search = GridSearchCV(model, param, scoring="neg_mean_squared_error", cv=cv, n_jobs=4, verbose=10)
    start=time()
    search.fit(X, y)
    print("RandomizedSearchCV took %.2f seconds for %d candidates parameter settings." % ((time() - start), n_iter_search))
    report(search)
    return search.best_estimator_

def cv_model(model,v,df,jobs=4):
    y=get_y(df)
    X=v.fit_transform(df)
    scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error',n_jobs=jobs)
    print(scores)
    print(scores.mean(),scores.std())

Для начала используем линейную регрессию, в качестве самой простой модели.

In [19]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge

lin_reg = Ridge()

# test_model(lin_reg,vec,df)
param = {"alpha":[0.001,0.1,1,2,3,5,10,100,1000]}
model = gs_model(lin_reg, param,vec,df)
cv_model(model,vec,df)
del lin_reg,model

/home/necromant/.virtualenvs/datascience/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/necromant/.virtualenvs/datascience/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/necromant/.virtualenvs/datascience/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/necromant/.virtualenvs/datascience/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversio

Fitting 5 folds for each of 9 candidates, totalling 45 fits


/home/necromant/.virtualenvs/datascience/lib/python3.6/site-packages/sklearn/model_selection/_search.py:271: UserWarning: The total space of parameters 9 is smaller than n_iter=20. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:    1.4s
[Parallel(n_jobs=4)]: Done   4 tasks      | elapsed:    1.5s
[Parallel(n_jobs=4)]: Done   9 tasks      | elapsed:    2.2s
[Parallel(n_jobs=4)]: Done  14 tasks      | elapsed:    2.6s
[Parallel(n_jobs=4)]: Done  21 tasks      | elapsed:    3.3s
[Parallel(n_jobs=4)]: Done  28 tasks      | elapsed:    4.0s
[Parallel(n_jobs=4)]: Done  37 tasks      | elapsed:    4.9s
[Parallel(n_jobs=4)]: Done  45 out of  45 | elapsed:    5.6s finished
/home/necromant/.virtualenvs/datascience/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: 

RandomizedSearchCV took 5.77 seconds for 20 candidates parameter settings.
Best param: {'alpha': 10}
Best score: -0.5176187512524733


/home/necromant/.virtualenvs/datascience/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/necromant/.virtualenvs/datascience/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[-0.51557751 -0.51594518 -0.52198992 -0.51647614 -0.51810501]
-0.5176187512524733 0.0023501735330393736


Попробуем улучшить результаты другой моделью

In [20]:
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor()
# test_model(knn,vec,df)
param = {"n_neighbors":[1,2,3,5,10,25,50,80,100],
        "leaf_size":[5,10,15,30,50,100],
        "metric":["minkowski","manhattan",]}
model = gs_model(knn, param,vec,df)
cv_model(model,vec,df)

del knn,model

/home/necromant/.virtualenvs/datascience/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/necromant/.virtualenvs/datascience/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/necromant/.virtualenvs/datascience/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/necromant/.virtualenvs/datascience/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversio

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:  5.3min
[Parallel(n_jobs=4)]: Done   4 tasks      | elapsed:  5.7min
[Parallel(n_jobs=4)]: Done   9 tasks      | elapsed: 16.2min
[Parallel(n_jobs=4)]: Done  14 tasks      | elapsed: 22.1min
[Parallel(n_jobs=4)]: Done  21 tasks      | elapsed: 33.6min
[Parallel(n_jobs=4)]: Done  28 tasks      | elapsed: 40.2min
[Parallel(n_jobs=4)]: Done  37 tasks      | elapsed: 53.8min
[Parallel(n_jobs=4)]: Done  46 tasks      | elapsed: 64.9min
[Parallel(n_jobs=4)]: Done  57 tasks      | elapsed: 80.2min
[Parallel(n_jobs=4)]: Done  68 tasks      | elapsed: 95.0min
[Parallel(n_jobs=4)]: Done  81 tasks      | elapsed: 112.6min
[Parallel(n_jobs=4)]: Done  94 tasks      | elapsed: 129.4min
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed: 136.1min finished


RandomizedSearchCV took 8316.89 seconds for 20 candidates parameter settings.
Best param: {'n_neighbors': 50, 'metric': 'manhattan', 'leaf_size': 5}
Best score: -0.31841112006475225


/home/necromant/.virtualenvs/datascience/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/necromant/.virtualenvs/datascience/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/necromant/.virtualenvs/datascience/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/necromant/.virtualenvs/datascience/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversio

[-0.31848366 -0.31802334 -0.31954974 -0.31771065 -0.3182882 ]
-0.31841112006475225 0.0006257569087413023


DecisionTree мне нравятся за их интерпретируемость до определенной глубины. Вдруг неплохо предскажет и можно заложить явным алгоритмом?

In [22]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
# test_model(tree_reg,vec,df)
param = {"max_depth":[None,2,5,10,15],
        "min_samples_split":[2,5,10,20,50,100]}
model = gs_model(tree_reg, param,vec,df)
cv_model(model,vec,df)

del tree_reg,model

/home/necromant/.virtualenvs/datascience/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/necromant/.virtualenvs/datascience/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/necromant/.virtualenvs/datascience/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/necromant/.virtualenvs/datascience/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversio

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:    2.4s
[Parallel(n_jobs=4)]: Done   4 tasks      | elapsed:    2.6s
[Parallel(n_jobs=4)]: Done   9 tasks      | elapsed:    6.8s
[Parallel(n_jobs=4)]: Done  14 tasks      | elapsed:    9.4s
[Parallel(n_jobs=4)]: Done  21 tasks      | elapsed:   13.2s
[Parallel(n_jobs=4)]: Done  28 tasks      | elapsed:   16.1s
[Parallel(n_jobs=4)]: Done  37 tasks      | elapsed:   19.7s
[Parallel(n_jobs=4)]: Done  46 tasks      | elapsed:   24.2s
[Parallel(n_jobs=4)]: Done  57 tasks      | elapsed:   29.0s
[Parallel(n_jobs=4)]: Done  68 tasks      | elapsed:   34.0s
[Parallel(n_jobs=4)]: Done  81 tasks      | elapsed:   41.3s
[Parallel(n_jobs=4)]: Done  94 tasks      | elapsed:   48.8s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:   49.8s finished


RandomizedSearchCV took 51.87 seconds for 20 candidates parameter settings.
Best param: {'min_samples_split': 50, 'max_depth': None}
Best score: -0.31561458180776736


/home/necromant/.virtualenvs/datascience/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/necromant/.virtualenvs/datascience/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/necromant/.virtualenvs/datascience/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/necromant/.virtualenvs/datascience/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversio

[-0.31544558 -0.31577905 -0.31595234 -0.31504462 -0.31585522]
-0.3156153622213402 0.00033240038146458665


Попробуем ансамблирование

In [25]:
from sklearn.ensemble import RandomForestRegressor

# rnd_reg = RandomForestRegressor(n_estimators=100, n_jobs=4)
rnd_reg = RandomForestRegressor(n_jobs=1)
# test_model(rnd_reg,vec,df)
param = {"n_estimators":[10,50,100,150,200],
        "max_depth":[2,3,5,10,15],
        "min_samples_split":[2,5,10,20,50,100],
        }
model = gs_model(rnd_reg, param,vec,df,jobs=4)
cv_model(model,vec,df,jobs=4)

del rnd_reg, model

/home/necromant/.virtualenvs/datascience/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/necromant/.virtualenvs/datascience/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/necromant/.virtualenvs/datascience/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/necromant/.virtualenvs/datascience/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversio

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:  1.2min
[Parallel(n_jobs=4)]: Done   4 tasks      | elapsed:  1.2min
[Parallel(n_jobs=4)]: Done   9 tasks      | elapsed:  6.2min
[Parallel(n_jobs=4)]: Done  14 tasks      | elapsed: 12.8min
[Parallel(n_jobs=4)]: Done  21 tasks      | elapsed: 18.6min
[Parallel(n_jobs=4)]: Done  28 tasks      | elapsed: 24.9min
[Parallel(n_jobs=4)]: Done  37 tasks      | elapsed: 26.1min
[Parallel(n_jobs=4)]: Done  46 tasks      | elapsed: 26.6min
[Parallel(n_jobs=4)]: Done  57 tasks      | elapsed: 27.8min
[Parallel(n_jobs=4)]: Done  68 tasks      | elapsed: 31.3min
[Parallel(n_jobs=4)]: Done  81 tasks      | elapsed: 36.1min
[Parallel(n_jobs=4)]: Done  94 tasks      | elapsed: 39.0min
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed: 40.0min finished


RandomizedSearchCV took 2665.89 seconds for 20 candidates parameter settings.
Best param: {'n_estimators': 200, 'min_samples_split': 20, 'max_depth': 15}
Best score: -0.3276899757561234


/home/necromant/.virtualenvs/datascience/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/necromant/.virtualenvs/datascience/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/necromant/.virtualenvs/datascience/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/necromant/.virtualenvs/datascience/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversio

[-0.32791668 -0.32827047 -0.32663361 -0.32713446 -0.32839712]
-0.32767046743447825 0.0006798578999119342


NameError: name 'rnd_clf' is not defined

In [29]:
import xgboost as xgb

# xgb_reg = xgb.XGBRegressor(nthread=4)
xgb_reg = xgb.XGBRegressor(nthread=4)
# test_model(xgb_reg,vec,df)
param = {"n_estimators":[50,100,150,200],
        "max_depth":[5,10,15],
        "learning_rate":[0.1,1,10],
         "reg_lambda":[0.1,1,10]
        }
model = gs_model(xgb_reg, param,vec,df,jobs=1)
cv_model(model,vec,df)

del xgb_reg,model

/home/necromant/.virtualenvs/datascience/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/necromant/.virtualenvs/datascience/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/necromant/.virtualenvs/datascience/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/necromant/.virtualenvs/datascience/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversio

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] reg_lambda=0.1, n_estimators=100, max_depth=15, learning_rate=1 .
[CV]  reg_lambda=0.1, n_estimators=100, max_depth=15, learning_rate=1, score=-0.3204156322601387, total= 1.0min
[CV] reg_lambda=0.1, n_estimators=100, max_depth=15, learning_rate=1 .


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.1min remaining:    0.0s


[CV]  reg_lambda=0.1, n_estimators=100, max_depth=15, learning_rate=1, score=-0.3204889703940479, total=  59.0s
[CV] reg_lambda=0.1, n_estimators=100, max_depth=15, learning_rate=1 .


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  2.2min remaining:    0.0s


...

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  7.5min remaining:    0.0s


[CV]  reg_lambda=1, n_estimators=100, max_depth=10, learning_rate=1, score=-0.30956015870388753, total=  37.2s
[CV] reg_lambda=1, n_estimators=100, max_depth=10, learning_rate=1 ...


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  8.2min remaining:    0.0s


[CV]  reg_lambda=1, n_estimators=100, max_depth=10, learning_rate=1, score=-0.30979141991828335, total=  36.0s
[CV] reg_lambda=10, n_estimators=200, max_depth=5, learning_rate=1 ...
[CV]  reg_lambda=10, n_estimators=200, max_depth=5, learning_rate=1, score=-0.3102606615833585, total=  36.6s
[CV] reg_lambda=10, n_estimators=200, max_depth=5, learning_rate=1 ...
[CV]  reg_lambda=10, n_estimators=200, max_depth=5, learning_rate=1, score=-0.3101493617604224, total=  36.6s
[CV] reg_lambda=10, n_estimators=200, max_depth=5, learning_rate=1 ...
[CV]  reg_lambda=10, n_estimators=200, max_depth=5, learning_rate=1, score=-0.310579490757417, total=  36.6s
[CV] reg_lambda=10, n_estimators=200, max_depth=5, learning_rate=1 ...
[CV]  reg_lambda=10, n_estimators=200, max_depth=5, learning_rate=1, score=-0.30995350045345443, total=  36.8s
[CV] reg_lambda=10, n_estimators=200, max_depth=5, learning_rate=1 ...
[CV]  reg_lambda=10, n_estimators=200, max_depth=5, learning_rate=1, score=-0.3100678283246621

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

Вылетает по непонятным причинам. лучшая модель получается reg_lambda=10, n_estimators=150, max_depth=10, learning_rate=1, score=-0.30511102674687024

In [25]:
xgb_reg = xgb.XGBRegressor(reg_lambda=10, n_estimators=150, max_depth=10, learning_rate=1)
cv_model(xgb_reg,vec,df)
del xgb_reg

/home/necromant/.virtualenvs/datascience/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/necromant/.virtualenvs/datascience/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/necromant/.virtualenvs/datascience/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/necromant/.virtualenvs/datascience/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversio

[-0.30519446 -0.30484437 -0.30599174 -0.30506423 -0.30511103]
-0.30524116290789144 0.000392741351677596


## Итог

наиболее удачной является модель xgb хоть и незначительно (RMSE = -0.305, против -0.315 у деревьем, например, но для соревнований это важно).

In [30]:
xgb_reg = xgb.XGBRegressor(reg_lambda=10, n_estimators=150, max_depth=10, learning_rate=1, nthread=4)
y = get_y(df)
X = vec.fit_transform(df)
X_test = vec.fit_transform(df_test)
xgb_reg.fit(X,y)
y_pred = xgb_reg.predict(X_test)
y_pred = np.exp(y_pred)-y_pred[np.where(y_pred<0)]=0

/home/necromant/.virtualenvs/datascience/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/necromant/.virtualenvs/datascience/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/necromant/.virtualenvs/datascience/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/necromant/.virtualenvs/datascience/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversio

array([3.5513668, 3.1374931, 2.874524 , ..., 2.966386 , 2.8441763,
       2.8862147], dtype=float32)

In [46]:
df_y_test = pd.DataFrame(df_test['test_id'].copy())
df_y_test['price'] = pd.Series(y_pred, index=df_y_test.index)
df_y_test.to_csv("submission.csv", index=False)

Загруженно на кагл. Очков значительно меньше.

Выводы:
 * по каглу:
      * внимательно читать подсчет очков. В данном случае RMSLE, а не RMSE
 * По моделе:
      * можно попробовать разделить категории товаров и использовать более распространенные
      * в описаниях товаров выделить метки и использовать их для улучшения работы
      * попробовать предварительно кластеризовать данные и учитывать кластеры в качестве признаков